# Build Your Own Model

One important feature of `orbit` is to allow users to build and customize some prototype models promptly to serve their own purpose. Users just need to code up the core model structure part, then orbit will facilitate and streamline the downstream functionalities, such as fit-predict, diagnostics, etc.

In this section, we give a demo on how to build up a new model, i.e., `PyroVIRegression`, with the help of orbit.

In [1]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist

import orbit
from orbit.models.template import BaseTemplate
from orbit.models.template import FullBayesianTemplate
from orbit.estimators.pyro_estimator import PyroEstimatorVI

from orbit.utils.simulation import make_regression

%matplotlib inline

In [2]:
assert orbit.__version__ == '1.0.13dev'

## Define a new model structure

In [3]:
class MyFitter:
    max_plate_nesting = 1  # max number of plates nested in model

    def __init__(self, data):
        for key, value in data.items():
            key = key.lower()
            if isinstance(value, (list, np.ndarray)):
                value = torch.tensor(value, dtype=torch.float)
            self.__dict__[key] = value

    def __call__(self):
        extra_out = {}
        
        p = self.regressor.shape[1]
        bias = pyro.sample("bias", dist.Normal(0, 1))
        weight = pyro.sample("weight", dist.Normal(0, 1).expand([p]).to_event(1))
        yhat = bias + weight @ self.regressor.transpose(-1, -2)
        obs_sigma = pyro.sample("obs_sigma", dist.HalfCauchy(self.response_sd))
        
        with pyro.plate("response_plate", self.num_of_obs):
            pyro.sample("response", dist.Normal(yhat, obs_sigma), obs=self.response)
        return extra_out

## Define data mapper

In [4]:
from enum import Enum

class MyDataMapper(Enum):
    NUM_OF_OBSERVATIONS = 'NUM_OF_OBS'
    RESPONSE = 'RESPONSE'
    RESPONSE_SD = 'RESPONSE_SD'
    REGRESSOR = 'REGRESSOR'

## Put it into the template

In [5]:
class BaseRegression(BaseTemplate):
    _fitter = MyFitter
    _data_input_mapper = MyDataMapper
    def __init__(self, regressor_col, **kwargs):
        super().__init__(**kwargs)  # create estimator in base class
        self.regressor_col = regressor_col
        self.regressor = None

    def _set_model_param_names(self):
        # sampling parameters
        self._model_param_names = ['bias', 'weight', 'obs_sigma']
    
    def _set_dynamic_attributes(self, df):
        # additional information want to bring into the model fitting
        self.regressor = df[self.regressor_col].values

class PyroVIRegression(FullBayesianTemplate, BaseRegression):
    
    _supported_estimator_types = [PyroEstimatorVI]
    
    def __init__(self, estimator_type=PyroEstimatorVI, **kwargs):
        super().__init__(estimator_type=estimator_type, **kwargs)

## Test out the new model!

Prepare the input data.

In [6]:
x, y, coefs = make_regression(120, [3.0, -1.0], bias=1.0)

In [7]:
df = pd.DataFrame(
    np.concatenate([y.reshape(-1, 1), x], axis=1), columns=['y', 'x1', 'x2']
)
df['week'] = pd.date_range(start='2016-01-04', periods=len(y), freq='7D')

In [8]:
df.head(5)

,y,x1,x2,week
0,1.863960,0.172792,0.000000,2016-01-04
1,2.317274,0.165219,-0.000000,2016-01-11
2,2.465284,0.452678,0.223187,2016-01-18
3,-0.593716,-0.000000,0.290559,2016-01-25
4,2.305148,0.182286,0.147066,2016-02-01


In [9]:
test_size = 20
train_df = df[:-test_size]
test_df = df[-test_size:]

Instantiate the new model object.

In [10]:
mod = PyroVIRegression(
    response_col='y', 
    date_col='week',
    regressor_col=['x1','x2'], 
    verbose=True,
    num_steps=501,
    seed=2021,
)

In [11]:
mod.fit(df=train_df)

INFO:root:Guessed max_plate_nesting = 2


step    0 loss = 21953, scale = 0.073408
step  100 loss = 12588, scale = 0.015333
step  200 loss = 12587, scale = 0.016251
step  300 loss = 12589, scale = 0.016101
step  400 loss = 12589, scale = 0.015745
step  500 loss = 12589, scale = 0.016569


In [12]:
estimated_weights = mod._posterior_samples['weight']

In [13]:
print("True Coef: {:.3f}, {:.3f}".format(coefs[0], coefs[1]) )
estimated_coef = np.median(estimated_weights, axis=0)
print("Estimated Coef: {:.3f}, {:.3f}".format(estimated_coef[0], estimated_coef[1]))

True Coef: 3.000, -1.000
Estimated Coef: 2.924, -0.934
